In [2]:
import paddle.fluid as fluid
from time import perf_counter
import numpy as np
import pandas as pd
import re
import jieba
from multiprocessing import cpu_count, Pool
from utils.pickle_io import *
import utils.config as config
import utils.wv_loader as wv_loader
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.word2vec import LineSentence
import logging

In [3]:
class Encoder(fluid.dygraph.Layer):
    def __init__(self, 
                 name_scope, 
                 enc_units, 
                 batch_size, 
                 vocab_size=1e5, 
                 word_vector=None,
                 param_attr=None,
                 bias_attr=None,
                 is_reverse=False,
                 gate_activation='sigmoid',
                 candidate_activation='tanh',
                 h_0=None,
                 origin_mode=False):
        '''
            Encoder初始化
            :param name_scope: 所在命名空间
            :param enc_units:GRU单元维度
            :param batch_size: Batch数量
            :param wordvector: 自定义词向量
            
        '''
        super(Encoder, self).__init__(name_scope)
        self.enc_units = int(enc_units)
        self.batch_size = int(batch_size)
        self.vocab_size = int(vocab_size)
        self.word_vector = word_vector
        
        # Embedding
        if self.word_vector is not None:
            self.vocab_size = int(self.word_vector.shape[0])
            w_param_attrs = fluid.ParamAttr(
                                name="emb_weight",
                                learning_rate=0.5,
                                initializer=fluid.initializer.NumpyArrayInitializer(self.word_vector),
                                trainable=True)
            self._embedding = fluid.dygraph.Embedding(
                                name_scope='embedding',
                                size=list(self.word_vector.shape),
                                param_attr= w_param_attrs,
                                is_sparse=False)
            # 如果有自定义词向量维度不符合 D*3，需要添加一层FC
            if self.word_vector.shape[1] != self.enc_units*3:
                self._fc = fluid.dygraph.FC('fc_for_gru', self.enc_units*3)
        else:
            self._embedding = fluid.dygraph.Embedding(
                                name_scope='embedding',
                                size=[self.vocab_size, self.enc_units*3],
                                param_attr='emb.w',
                                is_sparse=False)
        
        # GRU
        self._gru = fluid.dygraph.GRUUnit(
            self.full_name(),
            size=self.enc_units * 3,
            param_attr=param_attr,
            bias_attr=bias_attr,
            activation=candidate_activation,
            gate_activation=gate_activation,
            origin_mode=origin_mode)
        self.h_0 = h_0 if h_0 is not None else self.initialize_hidden_state()
        self.is_reverse = is_reverse
                                                
    
    def forward(self, inputs, hidden=None):
        '''
        调用Encoder时的计算
        :param inputs: variable类型的输入数据，维度（ N, T, D ）
        :param hidden: 隐藏层输入h_0
        :return output,state: output = hidden拼接向量，维度（ N, T, H ）
                              state = hidden时间维度的最后一个向量
        '''
        hidden = self.h_0 if hidden is None else hidden
        res = []
        for i in range(inputs.shape[1]):
            if self.is_reverse:
                i = inputs.shape[1] - 1 - i
            input_ = inputs[:, i:i + 1, :]
            input_ = fluid.layers.reshape(
                input_, [-1, input_.shape[2]], inplace=False)
            input_ = self._embedding(inputs)
            if hasattr(self, '_fc'):
                input_ = self._fc(input_)
            hidden, reset, gate = self._gru(input_, hidden)
            hidden_ = fluid.layers.reshape(
                hidden, [-1, 1, hidden.shape[1]], inplace=False)
            res.append(hidden_)
        if self.is_reverse:
            res = res[::-1]
        res = fluid.layers.concat(res, axis=1)
        return res, res[:,-1,:]
    
    def initialize_hidden_state(self):
        return fluid.layers.zeros((self.batch_size, self.enc_units), dtype='float32')

class BahdanauAttention(fluid.dygraph.Layer):
    def __init__(self, name_scope, units):
        super(BahdanauAttention, self).__init__(name_scope)
        self.W1 = fluid.dygraph.FC('attention_fc1', units, num_flatten_dims=2)
        self.W2 = fluid.dygraph.FC('attention_fc2', units, num_flatten_dims=2)
        self.V = fluid.dygraph.FC('attention_v', 1, num_flatten_dims=2)

    def forward(self, query, values):
        # query shape == (batch_size, 1, hidden_size)
        # values shape == (batch_size, max_length, hidden_size)
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        temp = (self.W1(values) + self.W2(query))
        score = self.V(fluid.layers.tanh(temp))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = fluid.layers.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        # paddlepaddle will not do broadcast autom atically, use elementwise_mul API
        context_vector = fluid.layers.elementwise_mul(values, attention_weights)
        context_vector = fluid.layers.reduce_sum(context_vector, dim=1)

        return context_vector, attention_weights

class Decoder(fluid.dygraph.Layer):
    def __init__(self, 
                 name_space, 
                 dec_units,
                 batch_size,
                 vocab_size=1e5, 
                 word_vector=None,
                 param_attr=None,
                 bias_attr=None,
                 is_reverse=False,
                 gate_activation='sigmoid',
                 candidate_activation='tanh',
                 h_0=None,
                 origin_mode=False):
        super(Decoder, self).__init__(name_space)
        self.batch_size = int(batch_size)
        self.dec_units = int(dec_units)
        self.vocab_size = int(vocab_size)
        self.word_vector = word_vector
        
       # Embedding
        if self.word_vector is not None:
            self.vocab_size = int(self.word_vector.shape[0])
            w_param_attrs = fluid.ParamAttr(
                                name="emb_weight",
                                learning_rate=0.5,
                                initializer=fluid.initializer.NumpyArrayInitializer(self.word_vector),
                                trainable=True)
            self._embedding = fluid.dygraph.Embedding(
                                name_scope='embedding',
                                size=list(self.word_vector.shape),
                                param_attr= w_param_attrs,
                                is_sparse=False)
        else:
            self._embedding = fluid.dygraph.Embedding(
                                name_scope='embedding',
                                size=[self.vocab_size, self.dec_units*3],
                                param_attr='emb.w',
                                is_sparse=False)
        
        # GRU
        self._gru = fluid.dygraph.GRUUnit(
            self.full_name(),
            size=self.dec_units * 3,
            param_attr=param_attr,
            bias_attr=bias_attr,
            activation=candidate_activation,
            gate_activation=gate_activation,
            origin_mode=origin_mode)
        
        # 如果维度不符合 D*3 不能传导，需要添加一层FC，保证维度是 D*3
        self._fc4gru = fluid.dygraph.FC('fc_for_gru', self.dec_units*3)
        
        # FC (N,H)==>(N,V)
        # self._fc = fluid.dygraph.FC('fc', self.vocab_size, act='softmax')
        self.is_reverse = is_reverse
        
    def forward(self, inputs, hidden, context_vector):
        # enc_output shape == (batch_size, max_length, hidden_size)
        # initial hidden is the last step of enc_output, hidden shape == (batch_size, hidden_size)
        inputs = self._embedding(inputs)
        # after concat shape is (batch_size, context_size + embedding_size)
        inputs = fluid.layers.concat([context_vector, inputs], axis=-1)
        inputs = self._fc4gru(inputs)
        
        hidden, reset, gate = self._gru(inputs, hidden)

        # output = self._fc(hidden)
        
        return hidden, gate

In [4]:
class Seq2Seq(fluid.dygraph.Layer):
    def __init__(self, name_scope, params):
        super(Seq2Seq, self).__init__(name_scope)
        self.embedding_matrix = wv_loader.load_embedding_matrix()
        self.params = params
        self.encoder = Encoder('encoder',
                               enc_units   = params["enc_units"],
                               batch_size  = params["batch_size"],
                               word_vector = self.embedding_matrix)

        self.attention = BahdanauAttention('attention', params["attn_units"])

        self.decoder = Decoder('decoder',
                               dec_units   = params["dec_units"],
                               batch_size  = params["batch_size"],
                               word_vector = self.embedding_matrix)

    def call_encoder(self, enc_inp):
        enc_hidden = self.encoder.initialize_hidden_state()
        enc_output, enc_hidden = self.encoder(enc_inp, enc_hidden)
        return enc_output, enc_hidden

    def call_decoder_onestep(self, dec_input, dec_hidden, enc_output):
        context_vector, attention_weights = self.attention(dec_hidden, enc_output)

        pred, dec_hidden = self.decoder(dec_input,
                                        None,
                                        context_vector)
        return pred, dec_hidden, context_vector, attention_weights

    def forward(self, dec_hidden, enc_output, dec_target):
        predictions = []
        attentions = []
        
        # print('enc_output shape:{}'.format(enc_output.shape))
        context_vector, attn = self.attention(fluid.layers.reshape(dec_hidden,[dec_hidden.shape[0],1,-1])
                                                    , enc_output)
        dec_input = fluid.layers.reshape(dec_target[:, 0], [dec_target.shape[0],1,-1])

        for t in range(1, dec_target.shape[1]):
            dec_hidden,_ = self.decoder(dec_input,
                                            dec_hidden,
                                            context_vector)

            context_vector, attn = self.attention(fluid.layers.reshape(dec_hidden,[dec_hidden.shape[0],1,-1])
                                                    , enc_output)
            # using teacher forcing
            dec_input = fluid.layers.reshape(dec_target[:, t], [dec_target.shape[0],1,-1])

            predictions.append(fluid.layers.reshape(dec_hidden, [dec_hidden.shape[0],1,-1]))
            attentions.append(fluid.layers.reshape(attn, [attn.shape[0],1,-1]))

        predictions = fluid.layers.concat(predictions, axis=1)
        attentions = fluid.layers.concat(attentions, axis=1)

        return predictions, attentions 

In [5]:
from utils.data_loader import load_dataset
from utils.wv_loader import *

Building prefix dict from the default dictionary ...
2019-12-24 19:28:01,453 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w3/yc8mtbd91vs80rp79zfgk8x00000gn/T/jieba.cache
2019-12-24 19:28:01,455 : DEBUG : Loading model from cache /var/folders/w3/yc8mtbd91vs80rp79zfgk8x00000gn/T/jieba.cache
Loading model cost 0.656 seconds.
2019-12-24 19:28:02,110 : DEBUG : Loading model cost 0.656 seconds.
Prefix dict has been built succesfully.
2019-12-24 19:28:02,112 : DEBUG : Prefix dict has been built succesfully.


In [7]:
import paddle

In [8]:
def loss_function(real, pred):
    # print('real shape:{}'.format(real.shape))
    # print('pred shape:{}'.format(pred.shape))
    # print('real:{}'.format(real.numpy()[0]))
    # print('pred:{}'.format(pred.numpy()[0]))
    # print('pred id of 0:{}'.format(np.argwhere(pred.numpy()[0]==0)))
    # 判断logit为1和0的数量
    real = fluid.layers.cast(real, dtype=np.int64)
    pad_array = fluid.layers.ones_like(real) * word2id['<PAD>']
    mask = fluid.layers.logical_not(fluid.layers.equal(real, pad_array))
    # print('mask:{}'.format(mask.numpy()[0]))
    # 计算decoder的长度
    dec_lens = fluid.layers.reduce_sum(fluid.layers.cast(mask, dtype=np.float64), dim=-1)
    # 计算loss值
    loss_ = fluid.layers.softmax_with_cross_entropy(logits=pred, label=real)
    # print('loss_:{}'.format(loss_.numpy()[0]))
    # 转换mask的格式
    mask = fluid.layers.cast(mask, dtype=loss_.dtype)
    # 调整loss
    loss_ *= mask
    # 确认下是否有空的摘要加入计算
    loss_ = fluid.layers.reduce_sum(loss_, dim=-1) / real.shape[0]
    # print('loss_ after reduce_sum:{}'.format(loss_.numpy()))
    return fluid.layers.reduce_mean(loss_)

In [9]:
epoch_num = 5
BATCH_SIZE = 32
X,Y,_ = load_dataset() 
# X = np.append(X[:,:201], X[:,-1].reshape((-1,1)), axis=1)
x_time_steps = X.shape[1]
y_time_steps = Y.shape[1]
Dataset = np.concatenate([X,Y], axis=1)
word2id, id2word = load_vocab()
params = {}
params["vocab_size"] = len(word2id)
params["embed_size"] = 300
params["enc_units"] = 512
params["attn_units"] = 16
params["dec_units"] = 512
params["batch_size"] = BATCH_SIZE
params["epochs"] = 5
params["max_enc_len"] = x_time_steps
params["max_dec_len"] = y_time_steps

# generator for paddle batch
def train_gen():
    for row in Dataset:
        yield row

with fluid.dygraph.guard():
    model = Seq2Seq('seq2seq', params)
    # Optimizer
    adam = fluid.optimizer.Adam(learning_rate=1e-5)
    train_reader = paddle.batch(
        train_gen, batch_size= BATCH_SIZE, drop_last=True)

    np.set_printoptions(precision=3, suppress=True)
    dy_param_init_value={}
    time_start = time_end = None # check elapsed time
    avg_loss = -1 # for print info
    for epoch in range(epoch_num):
        print('='*25 + 'Epoch {}'.format(epoch) + '='*25)
        for batch_id, data in enumerate(train_reader()):
            if batch_id % 10 == 0:
                if time_start is not None:
                    time_end = perf_counter()
                    print('Elapsed time:{:.2f}'.format(time_end - time_start))
                time_start = perf_counter()
                if avg_loss != -1:
                    print('completed batch {}, loss:{}'.format(batch_id, avg_loss.numpy()))
            data = np.array(data).astype('int64').flatten().reshape(BATCH_SIZE, x_time_steps+y_time_steps, 1)
            batch_x = data[:,:x_time_steps,:]
            batch_y = data[:,x_time_steps:,:]

            input = fluid.dygraph.to_variable(batch_x)
            label = fluid.dygraph.to_variable(batch_y)
            # print('input shape:{}'.format(input.shape))
            # print('label shape:{}'.format(label.shape))
            enc_output, enc_hidden = model.call_encoder(input)
            pred,_ = model(enc_hidden, enc_output, label)
            # print('pred shape:{}'.format(pred.shape))
            loss = loss_function(label[:, 1:], pred)
            # print('loss:{}'.format(loss.numpy()))
            avg_loss = fluid.layers.mean(loss)
            print('avg_loss:{}'.format(avg_loss.numpy()))
            try:
                avg_loss.backward()
            except Exception as e:
                print(e)
                print('X:{}'.format(batch_x))
                print('Y:{}'.format(batch_y))
            adam.minimize(avg_loss)
            model.clear_gradients()
            # for param in model.parameters():
            #     dy_param_init_value[param.name] = param.numpy()
            # model_states, _ = fluid.dygraph.load_dygraph("paddle_seq2seq")
            # model.set_dict(model_states)
        
            fluid.dygraph.save_dygraph(model.state_dict(), "paddle_seq2seq")
    
    
    # 检查是否成功保存模型
    restore = model.parameters()
    success = True
    for value in restore:
        if (not np.array_equal(value.numpy(), dy_param_init_value[value.name])) or (not np.isfinite(value.numpy().all())) or (np.isnan(value.numpy().any())):
            success = False
    print("model save and load success? {}".format(success))

=========================Epoch 0=========================


EnforceNotMet: 

--------------------------------------------
C++ Call Stacks (More useful to developers):
--------------------------------------------
0   std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > paddle::platform::GetTraceBackString<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&>(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&&&, char const*, int)
1   paddle::memory::detail::MetadataCache::load(paddle::memory::detail::MemoryBlock const*) const
2   paddle::memory::detail::MemoryBlock::split(paddle::memory::detail::MetadataCache*, unsigned long)
3   paddle::memory::detail::BuddyAllocator::SplitToAlloc(std::__1::__tree_const_iterator<std::__1::tuple<unsigned long, unsigned long, void*>, std::__1::__tree_node<std::__1::tuple<unsigned long, unsigned long, void*>, void*>*, long>, unsigned long)
4   paddle::memory::detail::BuddyAllocator::Alloc(unsigned long